In [1]:
import os,tarfile,sys
import ee
import geemap
import numpy as np
import pandas as pd
import folium
import copy
import math
from tqdm import trange,tqdm
from functools import partial
import geopandas as gpd
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

geemap version = 0.20.3
geemap path = ['D:\\Code_base\\anaconda\\envs\\GEE\\lib\\site-packages\\geemap']


# 定义数据

In [2]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))
    #projects/ee-mrwurenzhe/assets/Glacial_lake/Checkout_polygen

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [3]:
START_DATE  = ee.Date('2019-08-01')
END_DATE   = ee.Date('2019-08-30')
DEM = ee.Image('USGS/SRTMGL1_003')
models = ['volume', 'surface', None]
Buffer = 0
Model = models[0]
I = 900

AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(I))
s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filterBounds(AOI)
            .filterDate(START_DATE, END_DATE))
AOI_buffer = AOI.buffer(distance=400).bounds()

# 几何畸变定位与校正



In [4]:
# s1_col = slope_correction(s1_col, DEM, Model, buffer=Buffer)
# 分离升降轨
s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).first()
s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()

In [26]:
image = s1_descending
elevation = DEM
# get the image geometry and projection
geom = image.geometry()
proj = image.select(1).projection()

def getASCCorners(image, AOI_buffer):
    # 真实方位角
    coords = ee.Array(image.geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    maxLon = crdLons.sort().get(-1)
    minLat = crdLats.sort().get(0)
    maxLat = crdLats.sort().get(-1)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
               .multiply(180.0 / math.pi))

    if image.get('orbitProperties_pass').getInfo() == 'ASCENDING':
        azimuth = azimuth.add(270.0)
        rotationFromNorth = azimuth.subtract(360.0)
    elif s1_descending.get('orbitProperties_pass').getInfo() == 'DESCENDING':
        azimuth = azimuth.add(180.0)
        rotationFromNorth = azimuth.subtract(180.0)
    else:
        raise TypeError
    azimuthEdge = (ee.Feature(ee.Geometry.LineString([crdLons.get(crdLats.indexOf(minLat)), minLat, minLon,
                              crdLats.get(crdLons.indexOf(minLon))]), {'azimuth': azimuth}).copyProperties(image))

    coords = ee.Array(image.clip(
        AOI_buffer).geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    maxLon = crdLons.sort().get(-1)
    minLat = crdLats.sort().get(0)
    maxLat = crdLats.sort().get(-1)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
               .multiply(180.0 / math.pi))
    if image.get('orbitProperties_pass').getInfo() == 'ASCENDING':
        # 左上角
        startpoint = ee.List([minLon, maxLat])
        # 右下角 
        endpoint = ee.List([maxLon, minLat])
    elif s1_descending.get('orbitProperties_pass').getInfo() == 'DESCENDING':
        # 右上角
        startpoint = ee.List([maxLon, maxLat])
        # 左下角
        endpoint = ee.List([minLon, minLat])

    coordinates_dict = {'crdLons': crdLons, 'crdLats': crdLats,
                        'minLon': minLon, 'maxLon': maxLon, 'minLat': minLat, 'maxLat': maxLat}

    return azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict


Angle_aspect = ee.Terrain.aspect(image.select('angle'))
s1_azimuth_across = Angle_aspect.reduceRegion(
    ee.Reducer.mean(), geom, 1000).get('aspect')

azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict = getASCCorners(
    image, AOI_buffer)
trueAzimuth = azimuthEdge.get('azimuth')

# Correct the across-range-look direction
# s1_azimuth_across = ee.Number(s1_azimuth_across).add(rotationFromNorth)
s1_azimuth_across = ee.Number(trueAzimuth).subtract(90.0)
# s1_azimuth_across = ee.Number(trueAzimuth).subtract(90.0)
print("Corrected across-range-look direction", s1_azimuth_across.getInfo())
print("True azimuth across-range-look direction",
      ee.Number(trueAzimuth).subtract(90.0).getInfo())

theta_iRad = image.select('angle').multiply(np.pi / 180)  # 地面入射角度转为弧度
phi_iRad = ee.Image.constant(s1_azimuth_across).multiply(np.pi / 180)  # 方位角转弧度

alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
    np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡度(与地面夹角)
phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
    np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡向角，(坡度陡峭度)坡与正北方向夹角(陡峭度)，从正北方向起算，顺时针计算角度

height = elevation.setDefaultProjection(proj).clip(geom)

phi_rRad = phi_iRad.subtract(phi_sRad)     # (飞行方向角度-坡度陡峭度)飞行方向与坡向之间的夹角
# 分解坡度，在水平方向和垂直方向进行分解，为固定公式，cos对应水平分解，sin对应垂直分解
alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()     # 距离向分解
alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()     # 方位向分解

theta_liaRad = (alpha_azRad.cos().multiply(
    (theta_iRad.subtract(alpha_rRad)).cos())).acos()               # LIA
theta_liaDeg = theta_liaRad.multiply(180 / np.pi)                # LIA转弧度

# def LayoverAndShadow():

# ------------------------------RS几何畸变区域---------------------------------
layover = alpha_rRad.gt(theta_iRad).rename('layover')
ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
shadow = alpha_rRad.lt(
    ee.Image.constant(-1).multiply(ninetyRad.subtract(theta_iRad))).rename('shadow')

# # ------------------------------IJRS几何畸变区域-------------------------------
# layover = alpha_rRad.gt(theta_iRad).rename('layover')
# shadow = theta_liaDeg.gt(ee.Image.constant(85).multiply(np.pi / 180)).rename('shadow')

# # ------------------------------武大学报几何畸变区域---------------------------
# layover = theta_liaDeg.lt(ee.Image.constant(0).multiply(np.pi / 180)).rename('layover')
# shadow = theta_liaDeg.gt(ee.Image.constant(90).multiply(np.pi / 180)).rename('shadow')

# 纯alpha_rRad判断

# RIDEX


def rgbmask(layover, shadow):
    r = ee.Image.constant(0).select([0], ['red'])
    g = ee.Image.constant(0).select([0], ['green'])
    b = ee.Image.constant(0).select([0], ['blue'])

    r = r.where(layover, 255)
    g = g.where(shadow, 255)

    return ee.Image.cat([r, g, b]).byte().updateMask(image.mask())

# combine layover and shadow,因为shadow和layover都是0
no_data_mask = layover.And(shadow).rename('no_data_mask')
no_data_maskrgb = rgbmask(layover, shadow)

def Eq_pixels(x): return ee.Image.constant(0).where(x, x).updateMask(x.mask())

image2 = (Eq_pixels(image.select('VV')).rename('VV_sigma0').addBands(Eq_pixels(image.select('VH')).rename('VH_sigma0'))
          .addBands(Eq_pixels(image.select('angle')).rename('incAngle')).addBands(Eq_pixels(layover).rename('layover'))
          .addBands(Eq_pixels(shadow).rename('shadow'))
          .addBands(Eq_pixels(no_data_mask).rename('no_data_mask'))
          .addBands(no_data_maskrgb).addBands(Eq_pixels(height).rename('height')))

cal_image = (image2.addBands(ee.Image.pixelCoordinates(proj)).addBands(ee.Image.pixelLonLat())
             .reproject(crs=proj, scale=30).updateMask(image2.select('VV_sigma0').mask()).clip(AOI_buffer))
height = cal_image.select('height')
layover = cal_image.select('layover')
shadow = cal_image.select('shadow')
no_data_mask = cal_image.select('no_data_mask')
no_data_maskrgb = cal_image.select('no_data_maskrgb')

Corrected across-range-look direction 99.20386228776155
True azimuth across-range-look direction 99.20386228776155


In [27]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=8)
Auxiliarylines = ee.Geometry.LineString([startpoint,endpoint])
Map.addLayer(Auxiliarylines, {}, 'Auxiliarylines')
Map.addLayer(cal_image.select('VV_sigma0'),{'min':-40,'max':5}, 'VV_sigma0')
# Map.addLayer(cal_image.select('x'),{},'x')
Map.addLayer(cal_image.select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb')
Map # 显示地图窗口

Map(center=[28.98896597510626, 97.26846704471703], controls=(WidgetControl(options=['position', 'transparent_b…

In [16]:
# Map.addLayer(test.select('x'),{},'Templist')
# Map.addLayer(t.select('elevation').clip(Templist[0]), {}, 'final')
# Map.addLayer(Final.randomVisualizer(), {}, 'final4')
# Map.addLayer(RightLayover.randomVisualizer(), {}, 'RightLayover')
# Map.addLayer(LeftLayover.randomVisualizer(), {}, 'LeftLayover')
# Map.addLayer(Shadow.randomVisualizer(), {}, 'Shadow')

In [28]:
# 1、选一根线，逐点计算，获取梯度方向，构建梯度向量

# 2、根据已有的方位向量，构建y=kx+c的方程，使用该方程沿对角线像素进行遍历，计算路径上的被动几何畸变
# 将方位角转化为斜率
# 用于分析栅格像元
def reduce_tolist(each): return ee.Image(each).reduceRegion(
    reducer=ee.Reducer.toList(), geometry=image.geometry(), scale=30, maxPixels=1e13)

def angle2slope(angle):
    if type(angle) == ee.ee_number.Number:
        angle = angle.getInfo()
    if 0 < angle <= 90 or 180 < angle <= 270:
        if 180 < angle <= 270:
            angle = 90-(angle - 180)
        arc = angle/180*math.pi
        slop = math.tan(arc)
    elif 90 < angle <= 180 or 270 < angle <= 360:
        if 90 < angle <= 180:
            angle = angle - 90
        elif 270 < angle <= 360:
            angle = angle - 270
        arc = angle/180*math.pi
        slop = -math.tan(arc)
    return slop

# 计算斜率
K = angle2slope(s1_azimuth_across)

# 过Auxiliarylines中的点，从最小经度到最大经度
Max_Lon = coordinates_dict['maxLon'].getInfo()
Min_Lon = coordinates_dict['minLon'].getInfo()
AuxiliaryPoints = reduce_tolist(cal_image.select(
    ['height', 'longitude', 'latitude']).clip(Auxiliarylines))
Aux_lon = np.array(AuxiliaryPoints.get('longitude').getInfo())
Aux_lon, indices_Aux_lon = np.unique(Aux_lon, return_index=True)
Aux_lat = np.array(AuxiliaryPoints.get('latitude').getInfo())[indices_Aux_lon]

Templist = []
for X, Y in zip(Aux_lon, Aux_lat):
    C = Y - K*X
    Min_Lon_Y = K*Min_Lon + C
    Max_lon_Y = K*Max_Lon + C
    Templist.append(ee.Feature(ee.Geometry.LineString(
        [Min_Lon, Min_Lon_Y, Max_Lon, Max_lon_Y])))
SightLines = ee.FeatureCollection(Templist)
Map.addLayer(SightLines, {'color': 'white'}, 'test')

In [9]:
from scipy.signal import argrelextrema
from tqdm import tqdm

LPassive_layover_linList = []
RPassive_layover_linList = []
Shadow_linList = []
for eachLine in tqdm(Templist):
    # 求与辅助线相交的像素值,求取极值点位置
    LineImg = cal_image.select(
        ['height', 'layover', 'shadow', 'incAngle', 'x', 'y', 'longitude', 'latitude']).clip(eachLine)
    LineImg_point = LineImg.reduceRegion(
        reducer=ee.Reducer.toList(),
        geometry=image.geometry(),
        scale=30,
        maxPixels=1e13)

    # 排序算法,python版更迅速,使用 unique() 函数获取去重后的结果和对应索引
    PointDict = LineImg_point.getInfo()  # 需转换数据到本地，严重耗时
    order = np.argsort(PointDict['x'])
    PointDict = {k: np.array(v)[order] for k, v in PointDict.items()}
    PointDict['x'], PointDict['y'] = PointDict['x'] * \
        10, PointDict['y'] * 10  # 像素行列10m分辨率，由proj得

    # 寻找入射线上DEM的极大值点,Asending左侧可能出现layover，右侧可能出现shadow。   后期修补，考虑升降轨
    EasyIndex = lambda Data, Index, *Keys: [Data[key][Index] for key in Keys]
    index_max = argrelextrema(PointDict['height'], np.greater)[0]
    Angle_max, Z_max, X_max, Y_max = EasyIndex(
        PointDict, index_max, 'incAngle', 'height', 'x', 'y')

    # --------检索出一条线上所有Peak点的畸变
    LPassive_layover = []
    RPassive_layover = []
    Passive_shadow = []
    for each in range(len(index_max)):
        '''
        rx->r_angle , Peak点对应的数值
        Lay|L , 左侧叠掩
        Rlay ， 右侧叠掩
        '''
        rx = X_max[each]
        ry = Y_max[each]
        rh = Z_max[each]
        rindex = index_max[each]
        r_angle = Angle_max[each]

        # 被动叠掩（仅山底）
        if index_max[each] - 50 > 0:
            rangeIndex = range(index_max[each]-50, index_max[each])
        else:
            rangeIndex = range(0, index_max[each])

        L_h, L_x, L_y, L_lon, L_lat, L_angle = EasyIndex(
            PointDict, rangeIndex, 'height', 'x', 'y', 'longitude', 'latitude', 'incAngle')

        Llay_angle_iRad = np.arctan(
            (rh-L_h)/np.sqrt(np.square(L_x - rx) + np.square(L_y-ry)))
        Llay_angle = Llay_angle_iRad * 180 / math.pi
        index_Llay = np.where(Llay_angle > r_angle)[0]

        if len(index_Llay) != 0:
            tlon_Llay = L_lon[index_Llay]
            tlat_Llay = L_lat[index_Llay]
            LlayFeatureCollection = ee.FeatureCollection([ee.Feature(
                ee.Geometry.Point(x, y), {'values': 1}) for x, y in zip(tlon_Llay, tlat_Llay)])
            # 将属性值映射到图像上并设置默认值
            LPassive_layover.append(
                LlayFeatureCollection.reduceToImage(['values'], 'mean'))
            # image_with_values = image_with_values.paint(TempFeatureCollection,'values')

        # 阴影
        if index_max[each] + 50 < len(PointDict['x']):
            rangeIndex = range(index_max[each], index_max[each]+50)
        else:
            rangeIndex = range(index_max[each], len(PointDict['x']))

        R_h, R_x, R_y, R_lon, R_lat = EasyIndex(
            PointDict, rangeIndex, 'height', 'x', 'y', 'longitude', 'latitude')
        R_angle_iRad = np.arctan(
            (rh-R_h)/np.sqrt(np.square(R_x - rx) + np.square(R_y-ry)))
        R_angle = R_angle_iRad * 180 / math.pi
        index_Shadow = np.where(R_angle > (90-r_angle))[0]

        if len(index_Shadow) != 0:
            tlon_Shadow = R_lon[index_Shadow]
            tlat_Shadow = R_lat[index_Shadow]
            ShadowFeatureCollection = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(
                x, y), {'values': 3}) for x, y in zip(tlon_Shadow, tlat_Shadow)])
            Passive_shadow.append(
                ShadowFeatureCollection.reduceToImage(['values'], 'mean'))

        if len(index_Llay) != 0:
            # 被动叠掩(山顶右侧),与阴影运算交叉
            layoverM_x, layoverM_y, layoverM_h, layoverM_angle = \
                                        L_x[index_Llay[-1]], L_y[index_Llay[-1]], L_h[index_Llay[-1]], L_angle[index_Llay[-1]]  # 起算点
            Rlay_angle_iRad = np.arctan( 
                (R_h-layoverM_h)/np.sqrt(np.square(R_x - layoverM_x) + np.square(R_y-layoverM_y)))
            Rlay_angle = Rlay_angle_iRad * 180 / math.pi
            index_Rlayover = np.where(Rlay_angle > layoverM_angle)[0]
            if len(index_Rlayover) != 0:
                tlon_RLay = R_lon[index_Rlayover]
                tlat_RLay = R_lat[index_Rlayover]
                RLayFeatureCollection = ee.FeatureCollection([ee.Feature(
                    ee.Geometry.Point(x, y), {'values': 5}) for x, y in zip(tlon_RLay, tlat_RLay)])
                RPassive_layover.append(
                    RLayFeatureCollection.reduceToImage(['values'], 'mean'))

    if len(LPassive_layover) != 0:
        aggregated_image = ee.ImageCollection(
            LPassive_layover).mosaic().reproject(crs=proj, scale=30)
        LPassive_layover_linList.append(aggregated_image)

    if len(RPassive_layover) != 0:
        aggregated_image = ee.ImageCollection(
            RPassive_layover).mosaic().reproject(crs=proj, scale=30)
        RPassive_layover_linList.append(aggregated_image)

    if len(Passive_shadow) != 0:
        aggregated_image = ee.ImageCollection(
            Passive_shadow).mosaic().reproject(crs=proj, scale=30)
        Shadow_linList.append(aggregated_image)

LeftLayover = ee.ImageCollection(LPassive_layover_linList).mosaic()
RightLayover = ee.ImageCollection(RPassive_layover_linList).mosaic()
Shadow = ee.ImageCollection(Shadow_linList).mosaic()

100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [05:37<00:00,  4.17s/it]


In [14]:
RightLayover

In [13]:
Shadow

# 展示,生成左右对照Basemap与畸变检测对照

In [ ]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=12)
left_layer = geemap.ee_tile_layer(image.select('VV_sigma0'), {'min':-40,'max':5}, 'Origin')
right_layer = geemap.ee_tile_layer(image.select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb')
Map.split_map(left_layer,right_layer)
Map

In [ ]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=12)
Map.addLayer(height, {'min':3000,'max':5600}, 'DEM')
Map.addLayer(image.select('VV_sigma0'), {'min':-40,'max':5}, 'Origin')
Map.addLayer(image.select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb')
Map.addLayer
Map